In [2]:
import psycopg2
from IPython.display import display
import csv
import getpass
import pandas as pd

In [3]:
def create_database():
    # Database connection details
    username = 'postgres'
    password = getpass.getpass(prompt='Enter your PostgreSQL password: ')
    host = '127.16.2.21'
    initial_database = 'postgres'
    new_database = 'main'

    try:
        # Connect to the PostgreSQL server
        print(f"Connecting to {host} as {username}...")
        conn = psycopg2.connect(
            dbname=initial_database,
            user=username,
            password=password,
            host=host
        )

        cur = conn.cursor()

        # Set autocommit to True to execute commands outside a transaction block
        conn.set_session(autocommit=True)

        # Terminate any active connections to the 'project_ex' database
        cur.execute("""
        SELECT pg_terminate_backend(pid) 
        FROM pg_stat_activity 
        WHERE datname = %s AND pid <> pg_backend_pid();
        """, (new_database,))

        # Drop the 'project_ex' database if it exists
        cur.execute("DROP DATABASE IF EXISTS {}".format(new_database))

        # Create the 'project_ex' database
        cur.execute("CREATE DATABASE {}".format(new_database))

        # Close the initial connection
        conn.close()

        # Connect to the newly created 'project_ex' database
        conn = psycopg2.connect(
            dbname=new_database,
            user=username,
            password=password,
            host=host
        )

        cur = conn.cursor()

        return cur, conn

    except psycopg2.Error as e:
        print(f"Error: Unable to create database. {e.pgcode}: {e.pgerror}")
        return None, None

# Create the database
cur, conn = create_database()

# Check if database creation was successful
if cur and conn:
    print("Welcome! Database created successfully!")
else:
    print("Database creation failed.")


Enter your PostgreSQL password:  ········


Connecting to 127.16.2.21 as postgres...
Welcome! Database created successfully!


In [4]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.excute(query)
        conn.commit()

In [5]:
def create_tables(cur, conn):
    for query in drop_table_queries:
        cur.excute(query)
        conn.commit()

In [6]:
capsule = pd.read_csv(r"C:\Users\HP\Desktop\data\Capsule Hotel Risk Classification.csv")

In [7]:
capsule.head(2)

,Unnamed: 0,content,translated_content,priority_score,problem_domain
0,1,"Sorry, including the expensive price + apk usi...","Sorry, including the expensive price + apk usi...",2,0
1,2,Setelah install terus pas buka aplikasi ngebla...,"After installed, the app stuck with white display",3,1


In [8]:
capsule.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          700 non-null    int64 
 1   content             700 non-null    object
 2   translated_content  700 non-null    object
 3   priority_score      700 non-null    int64 
 4   problem_domain      700 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 27.5+ KB


In [9]:
capsule.rename(columns={'Unnamed: 0': 'nob'}, inplace=True)

In [10]:
cur, conn = create_database()

Enter your PostgreSQL password:  ········


Connecting to 127.16.2.21 as postgres...


In [11]:
capsule_table_insert = ("""
CREATE TABLE IF NOT EXISTS capsule(
    nob INT,
    content TEXT,
    translated_content TEXT,
    priority_score INT,
    problem_domain INT
);
""")

In [12]:
cur.execute(capsule_table_insert)
conn.commit()

### INSERT DATA INTO THE TABLE

In [13]:
capsule_table_insert = ("""INSERT INTO capsule(
nob ,
content ,
translated_content ,
priority_score ,
problem_domain)
VALUES (%s, %s, %s, %s, %s)    
""")

In [14]:
for i, row in capsule.iterrows():
    print(list(row))

[1, "Sorry, including the expensive price + apk using English for people who don't understand, it will be difficult to book", "Sorry, including the expensive price + apk using English for people who don't understand, it will be difficult to book", 2, 0]
[2, 'Setelah install terus pas buka aplikasi ngeblank putih', 'After installed, the app stuck with white display', 3, 1]
[3, 'gak bisa submit pada saat verifikasi data, padahal koneksi internet cepat & stabil', 'Cannot submit the data verification, even my internet is stable enough', 3, 1]
[4, 'Aplilasi restart mulu', 'The app keep restarting', 3, 1]
[5, 'udah mahal , makanan ga enak, banyak lalat', 'Overpriced, bad food, too many flies', 2, 0]
[6, 'fitur chat dengan host tidak bisa digunakan, padahal itu yg terpenting', "The chat feature which is important, can't be used", 2, 1]
[7, 'Mahal mahal.. kirain LBH murah dari refdorz dan oyo', "Overpriced, at first I think it's cheaper than refdorz and OYO", 2, 0]
[8, 'duh ini berisik amat yg

In [15]:
for i, row in capsule.iterrows():
    cur.execute(capsule_table_insert, list(row))

In [16]:
conn.commit()

In [17]:
cur.close()
conn.close()